# Distance and duration request using ORS

This notebook is an example for using different Openrouteservice tools. To get ready all needed packages were imported.

In [59]:
%%bash
git add ORS/use_cases/openrouteservice-examples/python/Kindergarten_SanFrancisco.ipynb

fatal: pathspec 'python/Kindergarten_SanFrancisco.ipynb' did not match any files


In [1]:
import folium
from folium import FeatureGroup, LayerControl, Map, Marker

from shapely import wkt, geometry
from shapely.geometry import shape

from pprint import pprint

import json

from openrouteservice import client, places, geocoding
from openrouteservice.directions import directions
from openrouteservice.isochrones import isochrones
from openrouteservice.places import places

We have just moved to San Francisco and are looking for the perfect location to get a new home. In a 15 minute walking radius there should be a kindergarten, a supertmarket and a post office. 

In [2]:
api_key = '58d904a497c67e00015b45fc2a6b6872037d44119582ef40cdf264c4'
clnt = client.Client(key=api_key)  

# isochrones

There are three different suggested locations for our new home. Let's visualize them and the 15 minute walking radius on a map:

In [3]:
map1 = folium.Map(tiles='Stamen Toner', location=([37.738684, -122.450523]), zoom_start=12) 

location_coords = {'first': [-122.430954, 37.792965], 'second': [-122.501636, 37.748653], 'third': [-122.446629, 37.736928]}

# Request of isochrones with 15 minute footwalk.
polygon_borders = []
for border_coords in location_coords:
    isochrone_request = isochrones(client=clnt, locations=location_coords[border_coords], profile='foot-walking', intervals=[900], segments=900)
    folium.GeoJson(isochrone_request).add_to(map1)
    polygon_borders.append(isochrone_request)
for house_coords in location_coords:
    location_coords[house_coords][0], location_coords[house_coords][1] = location_coords[house_coords][1], location_coords[house_coords][0]
    folium.Marker(location_coords[house_coords], icon=folium.Icon(color='lightgray',icon_color='#cc0000', icon='home', prefix='fa'), popup='Location suggestion').add_to(map1)
        
map1

# places

Now let's have a look how many kindergarten, supermarket and post offices there are in each walking radius. 

In [4]:
# Request of wished places via individual filter category id.
all_kindergarten = []
for kindergarten_request in polygon_borders:
    all_kindergarten += [clnt.places(request='pois', geojson=kindergarten_request['features'][0]['geometry'], filter_category_ids=[153], sortby='distance')]
        
all_supermarket = []
for supermarket_request in polygon_borders:
    all_supermarket += [clnt.places(request='pois', geojson=supermarket_request['features'][0]['geometry'], filter_category_ids=[518], sortby='distance')]

all_post = []
for post_request in polygon_borders:
    all_post += [clnt.places(request='pois', geojson=post_request['features'][0]['geometry'], filter_category_ids=[370], sortby='distance')]

print("Amount of kindergarten in the first, second and third suggested area: ", [len(kindergarten_amount['features']) for kindergarten_amount in all_kindergarten])
print("Amount of supermarket in the first, second and third suggested area: ", [len(supermarket_amount['features']) for supermarket_amount in all_supermarket])
print("Amount of post offices in the first, second and third suggested area: ", [len(post_amount['features']) for post_amount in all_post])

Amount of kindergarten in the first, second and third suggested area:  [1, 3, 1]
Amount of supermarket in the first, second and third suggested area:  [8, 1, 3]
Amount of post offices in the first, second and third suggested area:  [7, 0, 0]


Okay the first location looks pretty good: There is a kindergarten, eight supermarket and seven post offices. It is the perfect location for our new home!

# geocoding

To get representable names the geocoding api is a useful solution. 

In [5]:
name_markets = [(feat["properties"]['osm_tags']['name']) for feat in all_supermarket[0]['features']] # changing "Mollie Stone's" to "Mollie Stone"
name_markets[3] = 'Mollie Stone'
x = 0
for feat in all_supermarket[0]['features']:
    feat["properties"]['osm_tags']['name'] = name_markets[x]
    x = x+1
    
name_markets3 = [(feat["properties"]['osm_tags']['name']) for feat in all_supermarket[2]['features']] # changing "Mollie Stone's" to "Mollie Stone"
name_markets3[1] = 'Mollie Stone'
x = 0
for feat in all_supermarket[2]['features']:
    feat["properties"]['osm_tags']['name'] = name_markets3[x]
    x = x+1

In [44]:
all_places = {'all_kindergarten': all_kindergarten, 'all_supermarket': all_supermarket, 'all_post': all_post}

kindergarten_group = FeatureGroup(name='Kindergarten') 
supermarket_group = FeatureGroup(name='Supermarket')
post_group = FeatureGroup(name='Post Office')

for all_feat in all_places: # array mit allen locations und allen places
    all_features = all_places[all_feat]
    for group_feat in all_features: # array mit allen places
        single_feature = group_feat['features']
        for single_feat in single_feature:
            lon, lat = single_feat['geometry']['coordinates']
            if 'name' in single_feat['properties']:
                name = single_feat['properties']['osm_tags']['name']
            else:
                name = clnt.reverse_geocode(location=(lon, lat))['features'][0]['properties']['name']
                popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
            if '153' in single_feat['properties']['category_ids']:
                folium.map.Marker([lat, lon], icon=folium.Icon(color='lightgray', icon_color='#ffcc00', icon='child', prefix='fa'), popup=popup).add_to(kindergarten_group)
            elif '518' in single_feat['properties']['category_ids']:
                folium.map.Marker([lat, lon], icon=folium.Icon(color='lightgray', icon_color='#009900', icon='shopping-cart', prefix='fa'), popup=popup).add_to(supermarket_group)
            else:
                folium.map.Marker([lat, lon], icon=folium.Icon(color='lightgray', icon_color='#1a1aff', icon='envelope', prefix='fa'), popup=popup).add_to(post_group)

kindergarten_group.add_to(map1)
supermarket_group.add_to(map1)
post_group.add_to(map1)
                
map1

# directions

In [16]:
first_house_coords = [-122.430954, 37.792965]
first_area_direction = []
for all_feat in all_places: # array mit allen places für erste location
    first_features = all_places[all_feat][0]['features']
    for coords_first_features in first_features:
        first_area = [coords_first_features['geometry']['coordinates']]
        for coords in first_area:
            first_direction_request = directions(client=clnt, coordinates=(first_house_coords, coords), profile='foot-walking', format_out='geojson')
            folium.GeoJson(first_direction_request).add_to(kindergarten_group)
            first_area_direction.append(first_direction_request)

second_house_coords = [-122.501636, 37.748653]
for all_feat in all_places: # array mit allen places für erste location
    second_features = all_places[all_feat][1]['features']
    for coords_second_features in second_features:
        second_area = [coords_second_features['geometry']['coordinates']]
        for coords in second_area:
            second_direction_request = directions(client=clnt, coordinates=(second_house_coords, coords), profile='foot-walking', format_out='geojson')
            folium.GeoJson(second_direction_request).add_to(supermarket_group)
            
third_house_coords = [-122.446629, 37.736928]
for all_feat in all_places: # array mit allen places für erste location
    third_features = all_places[all_feat][2]['features']
    for coords_third_features in third_features:
        third_area = [coords_third_features['geometry']['coordinates']]
        for coords in third_area:
            third_direction_request = directions(client=clnt, coordinates=(third_house_coords, coords), profile='foot-walking', format_out='geojson')
            folium.GeoJson(third_direction_request).add_to(post_group)
        
map1.add_child(folium.map.LayerControl())   # funktioniert noch nicht         
map1

## Recommended walking route

To save time we want to create the optimal route. First bring the children to the kindergarten and then do all the groceries before heading home again. 

In [32]:
for wer in first_area_direction:
    eins = wer['features']
    for wer2 in eins:
        zwei = wer2['properties']['segments']
        for wer3 in zwei:
            pprint(wer3['duration'])

928.2
1023.6
689.9
1054.8
605.3
802.8
886.7
812.6
682.7
683.1
805
1018.8
664
773.6
1016.4
949.1


In [34]:
for wer in first_area_direction:
   # eins = wer['features']
    pprint(wer)

{'bbox': [-122.430948, 37.782942, -122.427724, 37.793103],
 'features': [{'geometry': {'coordinates': [[-122.430948, 37.792936],
                                            [-122.42964, 37.793103],
                                            [-122.429465, 37.792223],
                                            [-122.429305, 37.79143],
                                            [-122.429289, 37.791347],
                                            [-122.429274, 37.791275],
                                            [-122.429126, 37.790541],
                                            [-122.429111, 37.790467],
                                            [-122.429097, 37.790396],
                                            [-122.428944, 37.789639],
                                            [-122.428924, 37.789537],
                                            [-122.428905, 37.789443],
                                            [-122.428748, 37.788668],
                                 

                    'geometry_format': 'geojson',
                    'geometry_simplify': False,
                    'instructions': True,
                    'instructions_format': 'text',
                    'language': 'en',
                    'preference': 'fastest',
                    'profile': 'foot-walking',
                    'units': 'm'},
          'service': 'routing',
          'timestamp': 1524063534477},
 'type': 'FeatureCollection'}
{'bbox': [-122.430948, 37.790148, -122.423435, 37.793103],
 'features': [{'geometry': {'coordinates': [[-122.430948, 37.792936],
                                            [-122.42964, 37.793103],
                                            [-122.429465, 37.792223],
                                            [-122.429304, 37.792135],
                                            [-122.429291, 37.792128],
                                            [-122.429178, 37.792067],
                                            [-122.429066, 37.7920

 'features': [{'geometry': {'coordinates': [[-122.430948, 37.792936],
                                            [-122.431196, 37.792905],
                                            [-122.431286, 37.792894],
                                            [-122.431271, 37.792819],
                                            [-122.431124, 37.792088],
                                            [-122.431109, 37.792015],
                                            [-122.431094, 37.791943],
                                            [-122.430977, 37.79136],
                                            [-122.430946, 37.791208],
                                            [-122.430932, 37.791138],
                                            [-122.430917, 37.791066],
                                            [-122.430769, 37.790331],
                                            [-122.430754, 37.790257],
                                            [-122.430739, 37.790185],
                     

In [32]:
route_array = [] # create list with the needed elemets for the perfect route
route_array1.append({'geometry': {'coordinates': SF1_switched}}) 
route_array1.append(kindergarten1[0])
route_array1.append(supermarket1[2]) 
route_array1.append(post1[6]) 
route_array1.append({'geometry': {'coordinates': SF1_switched}}) 

route_array.append(first_house_coords, )

In [33]:
route_direct1 = {'coordinates': [(wer['geometry']['coordinates']) for wer in route_array1],
                  'profile': 'foot-walking',
                  'preference': 'shortest',
                  'geometry': 'true',
                  'format_out': 'geojson'
                 }

all_route1 = clnt.directions(**route_direct1) # request of directions

for alle in all_route1['features']:
    all_route = alle['geometry']['coordinates']
    
all_route =[[y,x] for x,y in all_route]

folium.PolyLine(all_route, color="#c6538c").add_to(route_group) # saving it up in the route feature group

route_group.add_to(map1)

Now we have the four feature groups with all the data saved up. Using now the Layer Control we can switch on and off the different sites and routes.

For completion we will calculate the total duration for the Optimal Route. We already received the duration with the direction module and saved it up in 'all_route1'.

In [35]:
routedur1 = [(alle['properties']['segments']) for alle in all_route1['features']]
routedur2 = [(wer['duration']) for wer in routedur1[0]] 
routedur2 # duration from site to site # home - kindergarten - post office - supermarket - home

[928.2, 431.1, 10.6, 683.1]

In [36]:
sum = 0 # sum of all single durations
for num in routedur2:
    sum += num
sum # duration in seconds

2053.0

In [37]:
summe = sum / 60 # dividing by 60 to get the duration in minutes
print("Duration for Optimal Route: {:.2f} minutes.".format(summe))

Duration for Optimal Route: 34.22 minutes.
